In [7]:

import pandas as pd

# Load your CSV
df = pd.read_csv("ridership_headline.csv")

# 1. Fill empty values (NaN) with 0
df = df.fillna(0)

# 2. Remove all bus columns
df = df.drop(columns=[c for c in df.columns if c.startswith("bus_")])

# 3. Combine ETS, Intercity, Komuter_Utara, Tebrau, and Komuter into one column
df["rail_komuter"] = (
    df["rail_ets"]
    + df["rail_intercity"]
    + df["rail_komuter_utara"]
    + df["rail_tebrau"]
    + df["rail_komuter"]
)

# 4. Drop the old columns (optional)
df = df.drop(columns=["rail_ets", "rail_intercity", "rail_komuter_utara", "rail_tebrau"])

df.rename(columns={
    "rail_lrt_ampang": "LRT Ampang/Sri Petaling Line",
    "rail_lrt_kj": "LRT Kelana Jaya Line",
    "rail_monorail": "KL Monorail",
    "rail_mrt_pjy": "MRT Putrajaya Line",
    "rail_mrt_kajang": "MRT Kajang Line",
    "rail_komuter": "KTM"
}, inplace=True)

# 1. Ensure date is parsed as datetime
df["date"] = pd.to_datetime(df["date"])

# 2. Extract year
df["year"] = df["date"].dt.year

# 3. Group by year and sum numeric columns
df_yearly = df.groupby("year", as_index=False).sum(numeric_only=True)

df_yearly = df_yearly.melt(
    id_vars=["year"],
    var_name="route_line",
    value_name="total_ridership"
)
df_yearly = df_yearly.pivot(
    index="route_line",
    columns="year",
    values="total_ridership"
)

df_yearly.columns = [f"ridership_{int(c)}" for c in df_yearly.columns]

df_yearly = df_yearly.reset_index()

# 4. Save to new file
df_yearly.to_csv("transport_yearly_sum.csv", index=False)



In [3]:
import pandas as pd

# Load your original daily CSV
df = pd.read_csv("ridership_headline.csv")

# Fill empty values with 0
df = df.fillna(0)

# Combine ETS, Intercity, Komuter_Utara, Tebrau, and Komuter into one commuter column
df["rail_komuter_total"] = (
    df["rail_ets"] +
    df["rail_intercity"] +
    df["rail_komuter_utara"] +
    df["rail_tebrau"] +
    df["rail_komuter"]
)

# Keep only the rail lines you want
df = df[["date","rail_lrt_ampang","rail_mrt_kajang","rail_lrt_kj","rail_monorail","rail_mrt_pjy","rail_komuter_total"]]

df.rename(columns={
    "rail_lrt_ampang": "LRT Ampang/Sri Petaling Line",
    "rail_lrt_kj": "LRT Kelana Jaya Line",
    "rail_monorail": "KL Monorail",
    "rail_mrt_pjy": "MRT Putrajaya Line",
    "rail_mrt_kajang": "MRT Kajang Line",
    "rail_komuter_total": "KTM"
}, inplace=True)

# Pivot longer so each row = route + day
df_long = df.melt(
    id_vars=["date"], 
    value_vars=["LRT Ampang/Sri Petaling Line","MRT Kajang Line","LRT Kelana Jaya Line","KL Monorail","MRT Putrajaya Line","KTM"],
    var_name="route_line", 
    value_name="ridership"
)

# Save CSV
df_long.to_csv("ridership_filtered.csv", index=False)
